In [ ]:
import pandas
import weave
weave.use_frontend_devmode()
from weave.ecosystem import openai
from weave.ecosystem import umap
from weave.ecosystem import hdbscan

In [ ]:
raw_data = pandas.read_csv('/Users/shawn/datasets/wandb_export_2023-06-03T15_01_20.066-07_00.csv')

data = weave.save(weave.ops.dataframe_to_arrow(raw_data), 'data')

In [ ]:
weave.panels.Board(
    vars={
        'data': data,
        'limit': 1005,
        'limited_data': lambda data, limit: data.limit(limit),
        'embeddings': lambda limited_data: openai.openai_embed(limited_data['question'], {"model": "text-embedding-ada-002"}),
        'clusterable_projection': lambda embeddings: umap.umap_project(
            embeddings, {
                'n_neighbors': 30,
                'min_dist': 0,
                'n_components': 2,
                #'random_state': 42
            }
        ),
        'clusters': lambda clusterable_projection: hdbscan.hdbscan_cluster(clusterable_projection, {
            'min_samples': 10,
            'min_cluster_size': 50
        }),
    },
    panels=[
        weave.panels.BoardPanel(
            lambda limited_data, clusterable_projection, clusters: weave.panels.Plot(
                limited_data,
                x=lambda row, index: clusterable_projection[index][0],
                y=lambda row, index: clusterable_projection[index][1],
                label=lambda row, index: clusters[index],
            )
        )
    ]
)